<a href="https://colab.research.google.com/github/Rossel/DataQuest_Courses/blob/master/028__Conditional_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COURSE 3/6: STORYTELLING THROUGH DATA VISUALIZATION

# MISSION 4: Conditional_plots

In this mission, we will explore how to ....

## 1. Introduction to Seaborn

So far, we've mostly worked with plots that are quick to analyze and make sense of. Line charts, scatter plots, and bar plots allow us to convey a few nuggets of insights to the reader. We've also explored how we can combine those plots in interesting ways to convey deeper insights and continue to extend the storytelling power of data visualization. In this mission, we'll explore how to quickly create multiple plots that are subsetted using one or more conditions.

We'll be working with the seaborn visualization library, which is built on top of matplotlib. [Seaborn](http://seaborn.pydata.org/) has good support for more complex plots, attractive default styles, and integrates well with the pandas library. Here are some examples of some complex plots that can be created using seaborn:

![Seaborn](https://s3.amazonaws.com/dq-content/seaborn_gallery.png)




## 2. Introduction to the Data Set

We'll be working with a data set of the passengers of the Titanic. The [Titanic shipwreck](https://en.wikipedia.org/wiki/Sinking_of_the_RMS_Titanic) is the most famous shipwreck in history and led to the creation of better safety regulations for ships. One substantial safety issue was that there were not enough lifeboats for every passenger on board, which meant that some passengers were prioritized over others to use the lifeboats.

The data set was compiled by [Kaggle](https://www.kaggle.com/c/titanic/data) for their introductory data science competition, called **Titanic: Machine Learning from Disaster**. The goal of the competition is to build machine learning models that can predict if a passenger survives from their attributes. 

The data for the passengers is contained in two files:

- `train.csv`: Contains data on 712 passengers
- `test.csv`: Contains data on 418 passengers

Each row in both data sets represents a passenger on the Titanic, and some information about them. We'll be working with the `train.csv` file, because the `Survived` column, which describes if a given passenger survived the crash, is preserved in the file. The column was removed in `test.csv`, to encourage competitors to practice making predictions using the data. Here are descriptions for each of the columns in `train.csv`:

- `PassengerId` -- A numerical id assigned to each passenger.
- `Survived` -- Whether the passenger survived (`1`), or didn't (`0`).
- `Pclass` -- The class the passenger was in.
- `Name` -- the name of the passenger.
- `Sex` -- The gender of the passenger -- male or female.
- `Age` -- The age of the passenger. Fractional.
- `SibSp` -- The number of siblings and spouses the passenger had on board.
- `Parch` -- The number of parents and children the passenger had on board.
- `Ticket` -- The ticket number of the passenger.
- `Fare` -- How much the passenger paid for the ticket.
- `Cabin` -- Which cabin the passenger was in.
- `Embarked` -- Where the passenger boarded the Titanic.

Here's what the first few rows look like: